In [24]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy import optimize

# Change the representation of float format to show the values as they are (Without wrong roundings).
pd.set_option('display.float_format', str)

class bond_pricer:
    
    def __init__(self, 
                 principal_value, 
                 coupon_rate,
                 settlement_date, 
                 maturity_date,
                 coupon_frequency=1,
                 convention='30/360',
                 yield_to_maturity=None, 
                 dirty_price=None,
                 clean_price=None):
        
        self.principal_value = principal_value
        self.coupon_rate = coupon_rate
        self.coupon_frequency = coupon_frequency
        self.coupon_amount = self.principal_value * (self.coupon_rate / self.coupon_frequency)
        self.settlement_date = datetime.strptime(settlement_date, '%d/%m/%Y').date()
        self.maturity_date = datetime.strptime(maturity_date, '%d/%m/%Y').date()
        self.convention = convention
        self.coupon_months = int(12 / self.coupon_frequency)  
        
        # Compute cash flow dates
        cflow_date = self.maturity_date
        cflow_dates = [self.maturity_date]
        while cflow_date + relativedelta(months=-self.coupon_months) > self.settlement_date:
            cflow_date += relativedelta(months=-self.coupon_months)
            cflow_dates.append(cflow_date)        
        self.cash_flow_dates = cflow_dates[::-1]
        
        # Compute accrued interest
        self.next_coupon_date = self.cash_flow_dates[0]
        self.previous_coupon_date = self.next_coupon_date + relativedelta(months=-self.coupon_months)
        if self.convention == 'ACT/ACT':
            self.accrued_days = (self.settlement_date - self.previous_coupon_date).days    
            self.curr_coupon_days = (self.next_coupon_date - self.previous_coupon_date).days
            self.accrual_period = self.accrued_days / self.curr_coupon_days
            self.accrued_interest = self.principal_value * (self.coupon_rate / self.coupon_frequency) * self.accrual_period
        elif self.convention == '30/360':
            d1 = min(30, self.previous_coupon_date.day)
            if d1 == 30:
                d2 = min(d1, self.settlement_date.day)
            else:
                d2 = self.settlement_date.day
            self.accrued_days = 360 * (self.settlement_date.year - self.previous_coupon_date.year) + 30 * (self.settlement_date.month - self.previous_coupon_date.month) + d2 - d1
            self.curr_coupon_days = 360 / self.coupon_frequency
            self.accrual_period = self.accrued_days / self.curr_coupon_days
            self.accrued_interest = self.principal_value * (self.coupon_rate / self.coupon_frequency) * self.accrual_period
        elif self.convention == 'ACT/360':
            self.accrued_days = (self.settlement_date - self.previous_coupon_date).days    
            self.curr_coupon_days = 360 / self.coupon_frequency
            self.accrual_period = self.accrued_days / self.curr_coupon_days
            self.accrued_interest = self.principal_value * (self.coupon_rate / self.coupon_frequency) * self.accrual_period
        elif self.convention == 'ACT/365':
            self.accrued_days = (self.settlement_date - self.previous_coupon_date).days
            self.curr_coupon_days = (self.next_coupon_date - self.previous_coupon_date).days
            self.accrual_period = self.accrued_days / 365
            self.accrued_interest = self.principal_value * (self.coupon_rate / self.coupon_frequency) * self.accrual_period
        
        # Compute cash flow amounts and time to cash flows and maturity
        first_tau = (self.curr_coupon_days - self.accrued_days) / self.curr_coupon_days        
        cash_flow_amounts = []
        taus = []
        for i in range(len(self.cash_flow_dates)):
            cash_flow_amounts.append(self.coupon_amount)
            taus.append((first_tau + i) / self.coupon_frequency)        
        cash_flow_amounts[-1] += self.principal_value
        self.cash_flows = np.array(cash_flow_amounts)
        self.taus = np.array(taus)
        self.time_to_maturity = self.taus[-1]
        
        # Compute clean and dirty prices
        if dirty_price:
            self.dirty_price = dirty_price
            self.clean_price = self.dirty_price - self.accrued_interest
        elif clean_price:
            self.clean_price = clean_price
            self.dirty_price = self.clean_price + self.accrued_interest
        elif yield_to_maturity:
            self.yield_to_maturity = yield_to_maturity
            self.ytm_adjusted = self.coupon_frequency*np.log(1 + self.yield_to_maturity / self.coupon_frequency)
            self.discount_factors = np.exp(-self.ytm_adjusted * self.taus)
            self.dirty_price = np.sum(self.cash_flows * self.discount_factors)
            self.clean_price = self.dirty_price - self.accrued_interest
        
        # Compute current yield
        self.current_yield = (self.coupon_rate * self.principal_value) / self.dirty_price
        
        # Compute yield to maturity
        if yield_to_maturity:
            self.yield_to_maturity = yield_to_maturity
            self.ytm_adjusted = self.coupon_frequency * np.log(1 + self.yield_to_maturity / self.coupon_frequency)
        else:
            def get_bond_price(yield_to_maturity, cash_flows, taus):
                return np.sum(cash_flows * np.exp(-yield_to_maturity * taus))
            
            get_ytm = lambda ytm: get_bond_price(ytm, self.cash_flows, self.taus) - self.dirty_price
            
            self.ytm_adjusted = optimize.newton(get_ytm, 0.04)
            self.yield_to_maturity = (np.exp(self.ytm_adjusted / self.coupon_frequency) - 1) * self.coupon_frequency
            self.discount_factors = np.exp(-self.ytm_adjusted * self.taus)
       
        # Compute duration
        self.first_derivative = np.sum(-self.taus * self.cash_flows * self.discount_factors)
        self.duration_macaulay = -1/self.dirty_price * self.first_derivative
        self.duration_modified = self.duration_macaulay / (1 + self.yield_to_maturity / self.coupon_frequency)                                                        
        
        # Compute convexity
        self.second_derivative = np.sum(self.taus**2 * self.cash_flows * self.discount_factors)
        self.convexity = 1/self.dirty_price * self.second_derivative
        
        # Generate summary table
        index = ['Dirty price','Clean price', 'Accrued interest', 'Face value', 'Coupon rate', 'Coupon frequency', 'Yield to maturity', 
                 'Time to maturity', 'Macaulay duration', 'Modified duration', 'Convexity', 'Settlement date', 'Maturity date', 'Convention']
        data = list(np.round([self.dirty_price, self.clean_price, self.accrued_interest], 7))
        data += list(np.round([self.principal_value, self.coupon_rate, self.coupon_frequency, 
                self.yield_to_maturity, self.time_to_maturity, self.duration_macaulay, self.duration_modified, self.convexity], 4))
        data += [self.settlement_date, self.maturity_date, self.convention]
        self.summary = pd.DataFrame(data, index, columns=[''])
        
    # Approximate new price after delta_ytm change in yield to maturity
    def approx_new_price(self, delta_ytm):
        return self.dirty_price * (1 + (-self.duration_modified * delta_ytm + 0.5 * self.convexity * delta_ytm**2))
    
    # Summary of the bond metrics
    def print_summary(self):
        print(self.summary)

SyntaxError: non-default argument follows default argument (4234472865.py, line 16)

In [64]:
# data = pd.read_csv("cleaned_data")
data = pd.read_excel("Dataset.xlsx")

In [68]:
data.rename({"Maturity date": "Maturity Date", "Coupon rate": "Coupon Rate (%)", 
             "Yield to maturity": "Yield to Maturity (%)", "Settlement date": "Settlement Date"
             },  axis=1, inplace=True)

data["Settlement Date"] = data["Settlement Date"].astype("str")
data["Maturity Date"] = data["Maturity Date"].astype("str")

data["Coupon Rate (%)"] = data["Coupon Rate (%)"] / 100
data["Yield to Maturity (%)"] = data["Yield to Maturity (%)"] / 100
data["Settlement Date"]  = data["Settlement Date"].apply(lambda x : '/'.join(x.split('-')[::-1]) )  
data["Maturity Date"] = data["Maturity Date"].apply(lambda x : '/'.join(x.split('-')[::-1])) 

In [ ]:
data

In [70]:
def show_dirty_prices(bonds):
    """ Shows the dirty prices of our bonds with calculated dirty prices
    
    Parameters:
        bonds (pd.DataFrame): pd.DataFrame object which represents our bonds data.
        
    Returns:
        dirty_prices_comparison (pd.DataFrame): pd.DataFrame object with 2 columns 'Real Dirty Price', 'Calculated Dirty Price'
    """
    
    calculated_dirty_prices = bonds.apply(lambda x : np.round(bond_pricer(principal_value=x["Principal Value"], coupon_rate=x["Coupon Rate (%)"], coupon_frequency=x["Coupon Frequency"], 
                    settlement_date=x["Settlement Date"], maturity_date=x["Maturity Date"], 
                    yield_to_maturity=x["Yield to Maturity (%)"], convention=x["Convention"]).dirty_price, 7), axis=1)
   
    # dirty_prices_comparison = pd.DataFrame({'Real Dirty Price': bonds['Dirty Price'], 'Calculated Dirty Price': calculated_dirty_prices})
    dirty_prices_comparison = pd.DataFrame({'Calculated Dirty Price': calculated_dirty_prices})
        
    return dirty_prices_comparison

In [71]:
show_dirty_prices(data)

,Calculated Dirty Price
0,73.9690045
1,84.4702124
2,95.3298039
3,129.9407458
4,111.4171478
...,...
999994,98.0601842
999995,99.9475383
999996,119.8618638
999997,102.2894049
